In [1]:
import pandas as pd

In [2]:
file_path = 'Resources/GDP_annual_growth.csv'

In [3]:
df = pd.read_csv(file_path)
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,NaN,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.881570e+10,NaN,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,NaN,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,NaN,NaN


In [4]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', 'Unnamed: 65'],
      dtype='object')

In [5]:
# grab desired columns
cleaned_df = df[['Country Name', '2019']].copy()
cleaned_df.head()

,Country Name,2019
0,Aruba,NaN
1,Afghanistan,1.929110e+10
2,Angola,8.881570e+10
3,Albania,1.527918e+10
4,Andorra,3.154058e+09


In [6]:
# grab country names of countries with nan
countries_test = cleaned_df.loc[cleaned_df['2019'].isnull()]['Country Name']

In [7]:
# since none printed we can drop nulls
cleaned_df.dropna(inplace= True)

In [8]:
billionaires = pd.read_csv('Resources/cleaned_data/cleaned_billionaires.csv')

In [9]:
# will print country name if gdp needed
list_need = []
for country in list(billionaires.Country):
    if (country not in list(cleaned_df['Country Name'])):
        list_need.append(country)
list_need = list(set(list_need))
list_need

['Guernsey',
 'Hong Kong',
 'South Korea',
 'Eswatini (Swaziland)',
 'Liechtenstein',
 'Egypt',
 'Russia',
 'Macao',
 'Monaco',
 'Taiwan',
 'Venezuela',
 'Czechia',
 'Slovakia']

In [10]:
list_need.remove('Guernsey')
list_need.remove('Taiwan')
list_need

['Hong Kong',
 'South Korea',
 'Eswatini (Swaziland)',
 'Liechtenstein',
 'Egypt',
 'Russia',
 'Macao',
 'Monaco',
 'Venezuela',
 'Czechia',
 'Slovakia']

In [11]:
# scrape for these countries
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import unidecode

base_url = "https://www.google.com/search?q=gdp+"
executable_path = {'executable_path':ChromeDriverManager().install()}



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\JordanRoessle\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [12]:
for country in list_need:
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(base_url + country)

    soup = BeautifulSoup(browser.html, "html.parser")
    gdp_to_add = soup.find("div", class_="ayqGOc kno-fb-ctx KBXm4e").text

    gdp_to_add = unidecode.unidecode(gdp_to_add)
    value = float(gdp_to_add.split(' ')[0])
    multiplier = gdp_to_add.split(' ')[1]
    if multiplier == 'trillion':
        value = value * 1000000000000
    elif multiplier == 'billion':
        value = value * 1000000000
    elif multiplier == 'million':
        value = value * 1000000
    cleaned_df = cleaned_df.append({'Country Name':country, 
                        '2019':value}, ignore_index=True)
    browser.quit()

In [13]:
# For simplicity we are manually adding Guernsey and Taiwan
cleaned_df = cleaned_df.append({'Country Name':'Guernsey', 
                        '2019':4615974000}, ignore_index=True)

cleaned_df = cleaned_df.append({'Country Name':'Taiwan', 
                        '2019':668510000000}, ignore_index=True)

In [14]:
# divide gdp by a million
cleaned_df['2019'] = cleaned_df['2019'] / 1000000

In [15]:
list_need = []
for country in list(billionaires.Country):
    if (country not in list(cleaned_df['Country Name'])):
        list_need.append(country)
list_need = list(set(list_need))
list_need

[]

In [16]:
# rename columns 
cleaned_df.columns = [['Country', 'GDP_in_millions']]

In [17]:
cleaned_df.head()

,Country,GDP_in_millions
0,Afghanistan,1.929110e+04
1,Angola,8.881570e+04
2,Albania,1.527918e+04
3,Andorra,3.154058e+03
4,Arab World,2.817415e+06


In [18]:
cleaned_df.to_csv('Resources/cleaned_data/cleaned_gdp_country.csv')